## RAG Microhack Challenge 3: Retrieve Data from Azure AI Search

__In this challenge, attendees will perform following tasks:__
1. Connect with Azure AI Search service
2. Querying of Azure AI Search vector index
3. Understanding of vectors
4. Retrieval of  document relevant to the search query
5. Understanding various search options (basic, semantic, vector & hybrid search)

__Pre-requisites:__
1. Azure OpenAI Service is created with text-embedding-ada-002 embedding model.
2. Azure AI Search service is created
3. Sample data is indexed in the AI Search service. HR Benefits sample data is provided in this repo.

### Step 1: Update ai_search.env file with your AI Search & AOAI service details. 

``Example ai_search.env file below.``

<img src="attachment:510cc22e-73d5-46a1-a20a-5791b889f38b.png" width=60% style="display: block; margin: auto;">


It is a good practive to put service configuration in a dot env file in Python. We will specify AI Search service configuration in ai_search.env file and use that to connect to the AI Search service you have created in the previous challenges

First we will install required packages for interacting with Azure AI Search.

In [ ]:
%%bash
pip install numpy
pip install python-dotenv
pip install azure-search-documents
pip install openai
pip install --upgrade pip

In [ ]:
from dotenv import dotenv_values

In [ ]:
# specify the name of the .env file name (default is ai_search.env)
env_filename = "ai_search.env"

# Load AI Search configuration (endpoint, key etc) from the environment file
env_config = dotenv_values(env_filename)

In [ ]:
# Import the environment variable values
ai_search_endpoint = env_config["AI_SEARCH_ENDPOINT"]
ai_search_key = env_config["AI_SEARCH_KEY"]
ai_search_index_name = env_config["AI_SEARCH_INDEX_NAME"]
ai_search_semantic_config_name = env_config["AI_SEARCH_SEMANTIC_CONFIG_NAME"]

In [ ]:
# Print the AI Search configuration values
print("AI Search service endpoint is: ", ai_search_endpoint, " and index name is: ", ai_search_index_name)

### Step 2: Connect to AI Search service

In [ ]:
# Import Required Python libraries for Azure AI Search and foundational packages
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient 

In [ ]:
ai_search_credential = AzureKeyCredential(ai_search_key)  

# Instantiate AI Search service client
ai_search_client = SearchClient(endpoint=ai_search_endpoint,  
                      index_name=ai_search_index_name,  
                      credential=ai_search_credential)  

### Step 3: Perform Basic Search using Azure AI Search service
Azure AI Search provides several options to customize your search queries. Here, we will explore basic, semantic, vector and hybrid search options.h:

In [ ]:
# Run basic search against your AI Search and print results
simple_query_results = ai_search_client.search(search_text="does my plan cover dental exam", top=3)  
  
for i, result in enumerate(simple_query_results):  
    print("****** SEARCH RESULT# :", i+1, " CONTENT ******", "Search score: ", result["@search.score"])
    ## Only printing 1000 characters from each matching document. Adjust as needed
    print(result["content"][0:1000]) 
    print("Source Document: ", result["filepath"], "\n" ) 

### Step 4: Perform Semantic Search using Azure AI Search service

Semantic search uses AI to understand the context of the search query and retrieve more relevant results.

In [ ]:
print("Running Semantic search using config: ", ai_search_semantic_config_name)
semantic_query_results = list(
        ai_search_client.search(
            search_text="does my plan cover dental exam",
            query_type="semantic",
            query_answer="extractive",
            query_answer_count=3,
            top=3,
            semantic_configuration_name=ai_search_semantic_config_name,
        )
    )

In [ ]:
for i, result in enumerate(semantic_query_results):  
    print("****** SEARCH RESULT#:", i+1, " CONTENT ******", "Search score:", 
          result["@search.score"], " Semantic Reranker Score:", result["@search.reranker_score"])
    ## Only printing 1000 characters from each matching document. Adjust as needed
    print(result["content"][0:100]) 
    print("Source Document: ", result["filepath"], "\n" ) 

### Step 5: Perform Vector Search using Azure AI Search service

``What Are Vectors? `` 
Vectors are mathematical representations of data points in a multi-dimensional space. In the context of Azure AI Search and machine learning, vectors are used to represent textual data, images, or other data types in a way that captures the inherent relationships and similarities between different data points. Each vector is typically a list of numbers (floats) that define the position of a data point in the vector space.

In [ ]:
from openai import AzureOpenAI
import requests
import json

In [ ]:
aoai_api_key = env_config["AOAI_KEY"]
aoai_endpoint = env_config["AOAI_ENDPOINT"]
aoai_embedding_deployment_name= env_config["AOAI_EMBEDDING_DEPLOYMENT_NAME"]

In [ ]:
# Initialize the Azure OpenAI client
aoai_client = AzureOpenAI(
  api_key = aoai_api_key,  
  api_version = "2024-02-01",
  azure_endpoint = aoai_endpoint
)

In [ ]:
# 1. Create a vector embedding for a query using Azure OpenAI embedding API
search_text = "does my plan cover dental exam"
search_text_embedding = aoai_client.embeddings.create(input = [search_text], model=aoai_embedding_deployment_name).data[0].embedding

In [ ]:
# Here is how the query "does my plan cover dental exam" looks when converted to a vector
print("Length of the vector embeddings: ", len(search_text_embedding))
print(search_text_embedding)

#### Note on embeddings size
The embedding vector size depends on the model used to create embedding. We are using OpenAI "text-embedding-ada-002" model which gives vectors of dimension 1536. You can see (below) Azure AI Search index has "contentVector" field of the same dimension.

<img src="attachment:65bc7ef2-6e39-41de-a869-52a8fa99fd41.png" width=70% style="display: block; margin: auto;">


##### Perform Vector search using the search query embeddings

In [ ]:
endpoint = f"{ai_search_endpoint}/indexes/{ai_search_index_name}/docs/search?api-version=2023-11-01"

# Headers for the request
headers = {
    "Content-Type": "application/json",
    "api-key": ai_search_key,
}

# Payload for the vector search
payload = {
        "count": True,
        "vectorQueries": [
            {
                "vector": search_text_embedding,
                "k": 5,
                "fields": "contentVector",
                "kind": "vector",
                "exhaustive": True
            }
        ]
    }

In [ ]:
# Perform vector search - Note that you are just sending vector embeddings as the search data
response = requests.post(endpoint, headers=headers, data=json.dumps(payload))

In [ ]:
# Check for successful response
if response.status_code == 200:
    results = response.json()
    print("Search results:")
    for result in results.get("value", []):
        print(result)
else:
    print(f"Error: {response.status_code}, {response.text}")

#### [OPTIONAL] Step 6: Semantic Search via REST API to get @semantic.answer

In [ ]:
import requests
import json

url = f"{ai_search_endpoint}/indexes/{ai_search_index_name}/docs"
headers = {
    "Content-Type": "application/json",
    "api-key": ai_search_key
}
params = {
    "api-version": "2024-09-01-preview",
    "search": "does my plan cover eye exams",
    "queryType": "semantic",
    "answers": "extractive|count-3",
    "semanticConfiguration": ai_search_semantic_config_name,  
    "top": 3  
}

response = requests.get(url, headers=headers, params=params)
data = response.json()
print(json.dumps(data, indent=4))

In [ ]:
# Print answers provided by the semantic search
data["@search.answers"]

In [ ]:
def search_ai_search(query_text, top=3):
    url = f"{ai_search_endpoint}/indexes/{ai_search_index_name}/docs"
    headers = {
        "Content-Type": "application/json",
        "api-key": ai_search_key
    }
    params = {
        "api-version": "2024-09-01-preview",
        "search": query_text,
        "queryType": "semantic",
        "answers": "extractive|count-3",
        "semanticConfiguration": ai_search_semantic_config_name,  
        "top": top  
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return None
    


## Interactive Vector Search
Update the below 'query_text' value to see how the vector search response to different queries.

### **Important:** 
The below results are the results that the Semantic Kernal (orchestrator) receives. It then passes the top N results to the Azure OpenAI model, and allows the model to process the results. The model makes sense of the results before sending a more human-like chat like response to the user.

### Example Flow:
User input text -> Semantic Kernal -> Model -> Semantic Kernal -> Vector Index -> Vector Search Results -> Semantic Kernal -> Model -> User output response text

In [ ]:
# Test the function with a query
# OPTIONAL: Change the query_text to test with a different query
query_text = "does my plan cover dental exam"

result = search_ai_search(query_text)
print(json.dumps(result, indent=4))